In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
    eloss += tf.reduce_mean(targetQs) # minimize Q
    aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
    eloss += tf.reduce_mean(eQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.6842 eloss:5.2008 aloss2:2.0389 exploreP:0.9984
Episode:1 meanR:23.0000 R:30.0000 rate:0.0600 aloss:0.6839 eloss:5.1570 aloss2:2.0683 exploreP:0.9955
Episode:2 meanR:21.6667 R:19.0000 rate:0.0380 aloss:0.6843 eloss:5.1157 aloss2:2.0405 exploreP:0.9936
Episode:3 meanR:26.0000 R:39.0000 rate:0.0780 aloss:0.6847 eloss:5.0930 aloss2:2.0294 exploreP:0.9898
Episode:4 meanR:31.6000 R:54.0000 rate:0.1080 aloss:0.6831 eloss:5.0705 aloss2:1.9646 exploreP:0.9845
Episode:5 meanR:31.3333 R:30.0000 rate:0.0600 aloss:0.6917 eloss:5.0264 aloss2:1.9748 exploreP:0.9816
Episode:6 meanR:28.8571 R:14.0000 rate:0.0280 aloss:0.6845 eloss:4.9872 aloss2:1.9671 exploreP:0.9802
Episode:7 meanR:28.0000 R:22.0000 rate:0.0440 aloss:0.6812 eloss:4.9982 aloss2:1.9049 exploreP:0.9781
Episode:8 meanR:27.7778 R:26.0000 rate:0.0520 aloss:0.6851 eloss:4.9703 aloss2:1.9317 exploreP:0.9756
Episode:9 meanR:27.1000 R:21.0000 rate:0.0420 aloss:0.6872 eloss:4.9386 aloss2:1.8

Episode:80 meanR:26.7778 R:16.0000 rate:0.0320 aloss:0.7336 eloss:4.3179 aloss2:1.6663 exploreP:0.8070
Episode:81 meanR:26.6098 R:13.0000 rate:0.0260 aloss:0.7217 eloss:4.2965 aloss2:1.7177 exploreP:0.8059
Episode:82 meanR:26.5422 R:21.0000 rate:0.0420 aloss:0.7210 eloss:4.3135 aloss2:1.6869 exploreP:0.8043
Episode:83 meanR:26.8095 R:49.0000 rate:0.0980 aloss:0.7343 eloss:4.3103 aloss2:1.7338 exploreP:0.8004
Episode:84 meanR:26.7647 R:23.0000 rate:0.0460 aloss:0.7219 eloss:4.3024 aloss2:1.7185 exploreP:0.7986
Episode:85 meanR:26.6977 R:21.0000 rate:0.0420 aloss:0.7239 eloss:4.3102 aloss2:1.6927 exploreP:0.7969
Episode:86 meanR:26.5402 R:13.0000 rate:0.0260 aloss:0.7393 eloss:4.3037 aloss2:1.7491 exploreP:0.7959
Episode:87 meanR:26.5114 R:24.0000 rate:0.0480 aloss:0.7264 eloss:4.2990 aloss2:1.7766 exploreP:0.7940
Episode:88 meanR:26.6292 R:37.0000 rate:0.0740 aloss:0.7502 eloss:4.3085 aloss2:1.7595 exploreP:0.7911
Episode:89 meanR:26.5333 R:18.0000 rate:0.0360 aloss:0.7352 eloss:4.3044 

Episode:159 meanR:33.2800 R:20.0000 rate:0.0400 aloss:0.7697 eloss:4.2552 aloss2:2.1633 exploreP:0.6220
Episode:160 meanR:33.2300 R:22.0000 rate:0.0440 aloss:0.7645 eloss:4.2316 aloss2:2.1650 exploreP:0.6206
Episode:161 meanR:33.2400 R:21.0000 rate:0.0420 aloss:0.7154 eloss:4.2461 aloss2:2.1418 exploreP:0.6194
Episode:162 meanR:33.6000 R:59.0000 rate:0.1180 aloss:0.7170 eloss:4.2269 aloss2:2.1794 exploreP:0.6158
Episode:163 meanR:33.8100 R:35.0000 rate:0.0700 aloss:0.7330 eloss:4.2223 aloss2:2.1903 exploreP:0.6137
Episode:164 meanR:34.2100 R:60.0000 rate:0.1200 aloss:0.7377 eloss:4.2303 aloss2:2.1736 exploreP:0.6100
Episode:165 meanR:34.0100 R:18.0000 rate:0.0360 aloss:0.7413 eloss:4.2227 aloss2:2.1995 exploreP:0.6090
Episode:166 meanR:34.7100 R:103.0000 rate:0.2060 aloss:0.7444 eloss:4.2341 aloss2:2.1667 exploreP:0.6028
Episode:167 meanR:34.5600 R:18.0000 rate:0.0360 aloss:0.7516 eloss:4.2254 aloss2:2.1870 exploreP:0.6018
Episode:168 meanR:34.5900 R:25.0000 rate:0.0500 aloss:0.7848 el

Episode:238 meanR:41.5400 R:33.0000 rate:0.0660 aloss:0.7401 eloss:4.2377 aloss2:2.1717 exploreP:0.4481
Episode:239 meanR:41.6700 R:38.0000 rate:0.0760 aloss:0.7635 eloss:4.2370 aloss2:2.1742 exploreP:0.4465
Episode:240 meanR:41.5600 R:33.0000 rate:0.0660 aloss:0.7451 eloss:4.2337 aloss2:2.1716 exploreP:0.4450
Episode:241 meanR:41.6800 R:37.0000 rate:0.0740 aloss:0.7395 eloss:4.2382 aloss2:2.1604 exploreP:0.4434
Episode:242 meanR:41.3800 R:50.0000 rate:0.1000 aloss:0.7816 eloss:4.2462 aloss2:2.1551 exploreP:0.4413
Episode:243 meanR:41.5000 R:34.0000 rate:0.0680 aloss:0.7546 eloss:4.2369 aloss2:2.1744 exploreP:0.4398
Episode:244 meanR:41.4300 R:25.0000 rate:0.0500 aloss:0.7313 eloss:4.2394 aloss2:2.1670 exploreP:0.4387
Episode:245 meanR:41.5200 R:37.0000 rate:0.0740 aloss:0.7219 eloss:4.2435 aloss2:2.1665 exploreP:0.4371
Episode:246 meanR:42.2900 R:95.0000 rate:0.1900 aloss:0.7472 eloss:4.2394 aloss2:2.1879 exploreP:0.4331
Episode:247 meanR:42.3600 R:49.0000 rate:0.0980 aloss:0.7272 elo

Episode:317 meanR:46.1500 R:58.0000 rate:0.1160 aloss:0.7737 eloss:4.2145 aloss2:2.1621 exploreP:0.3118
Episode:318 meanR:46.8300 R:88.0000 rate:0.1760 aloss:0.7871 eloss:4.2176 aloss2:2.1684 exploreP:0.3092
Episode:319 meanR:47.3400 R:98.0000 rate:0.1960 aloss:0.7584 eloss:4.2206 aloss2:2.1903 exploreP:0.3063
Episode:320 meanR:47.4300 R:45.0000 rate:0.0900 aloss:0.7594 eloss:4.2171 aloss2:2.1663 exploreP:0.3049
Episode:321 meanR:48.1700 R:106.0000 rate:0.2120 aloss:0.7463 eloss:4.2237 aloss2:2.1541 exploreP:0.3018
Episode:322 meanR:48.1600 R:34.0000 rate:0.0680 aloss:0.7710 eloss:4.2172 aloss2:2.1467 exploreP:0.3008
Episode:323 meanR:48.2000 R:29.0000 rate:0.0580 aloss:0.7716 eloss:4.2128 aloss2:2.1485 exploreP:0.3000
Episode:324 meanR:49.0600 R:106.0000 rate:0.2120 aloss:0.7710 eloss:4.2194 aloss2:2.1374 exploreP:0.2969
Episode:325 meanR:48.9000 R:55.0000 rate:0.1100 aloss:0.7465 eloss:4.2158 aloss2:2.1411 exploreP:0.2954
Episode:326 meanR:48.6800 R:58.0000 rate:0.1160 aloss:0.7635 e

...

Episode:1186 meanR:48.3500 R:29.0000 rate:0.0580 aloss:0.5800 eloss:4.2231 aloss2:2.1877 exploreP:0.0141
Episode:1187 meanR:48.4200 R:39.0000 rate:0.0780 aloss:0.5525 eloss:4.2268 aloss2:2.1901 exploreP:0.0141
Episode:1188 meanR:48.1200 R:30.0000 rate:0.0600 aloss:0.5747 eloss:4.2243 aloss2:2.1921 exploreP:0.0140
Episode:1189 meanR:48.1000 R:40.0000 rate:0.0800 aloss:0.5922 eloss:4.2162 aloss2:2.1970 exploreP:0.0140
Episode:1190 meanR:47.9100 R:29.0000 rate:0.0580 aloss:0.6373 eloss:4.2174 aloss2:2.1961 exploreP:0.0140
Episode:1191 meanR:47.7500 R:34.0000 rate:0.0680 aloss:0.5711 eloss:4.2275 aloss2:2.1821 exploreP:0.0140
Episode:1192 meanR:48.2800 R:85.0000 rate:0.1700 aloss:0.5626 eloss:4.2277 aloss2:2.1788 exploreP:0.0140
Episode:1193 meanR:48.2000 R:59.0000 rate:0.1180 aloss:0.5679 eloss:4.2274 aloss2:2.1724 exploreP:0.0139
Episode:1194 meanR:48.3500 R:59.0000 rate:0.1180 aloss:0.5748 eloss:4.2257 aloss2:2.1754 exploreP:0.0139
Episode:1195 meanR:48.3700 R:46.0000 rate:0.0920 aloss:

Episode:1265 meanR:47.2100 R:41.0000 rate:0.0820 aloss:0.5524 eloss:4.2312 aloss2:2.1919 exploreP:0.0128
Episode:1266 meanR:46.9000 R:49.0000 rate:0.0980 aloss:0.5337 eloss:4.2344 aloss2:2.1841 exploreP:0.0128
Episode:1267 meanR:46.9000 R:32.0000 rate:0.0640 aloss:0.6119 eloss:4.2165 aloss2:2.1999 exploreP:0.0128
Episode:1268 meanR:46.8700 R:34.0000 rate:0.0680 aloss:0.5801 eloss:4.2234 aloss2:2.1787 exploreP:0.0128
Episode:1269 meanR:46.7000 R:32.0000 rate:0.0640 aloss:0.5995 eloss:4.2191 aloss2:2.1935 exploreP:0.0128
Episode:1270 meanR:46.7300 R:37.0000 rate:0.0740 aloss:0.5346 eloss:4.2436 aloss2:2.1483 exploreP:0.0127
Episode:1271 meanR:46.9300 R:62.0000 rate:0.1240 aloss:0.5724 eloss:4.2278 aloss2:2.1852 exploreP:0.0127
Episode:1272 meanR:47.0300 R:42.0000 rate:0.0840 aloss:0.5879 eloss:4.2203 aloss2:2.1792 exploreP:0.0127
Episode:1273 meanR:46.6600 R:34.0000 rate:0.0680 aloss:0.5438 eloss:4.2266 aloss2:2.1938 exploreP:0.0127
Episode:1274 meanR:47.0800 R:71.0000 rate:0.1420 aloss:

Episode:1344 meanR:49.1200 R:32.0000 rate:0.0640 aloss:0.5406 eloss:4.2380 aloss2:2.2012 exploreP:0.0119
Episode:1345 meanR:48.6400 R:34.0000 rate:0.0680 aloss:0.5378 eloss:4.2369 aloss2:2.2018 exploreP:0.0119
Episode:1346 meanR:48.8800 R:58.0000 rate:0.1160 aloss:0.5553 eloss:4.2264 aloss2:2.1991 exploreP:0.0119
Episode:1347 meanR:48.9300 R:53.0000 rate:0.1060 aloss:0.5458 eloss:4.2335 aloss2:2.1990 exploreP:0.0119
Episode:1348 meanR:48.9000 R:32.0000 rate:0.0640 aloss:0.5129 eloss:4.2303 aloss2:2.2067 exploreP:0.0119
Episode:1349 meanR:48.9200 R:32.0000 rate:0.0640 aloss:0.5321 eloss:4.2444 aloss2:2.1889 exploreP:0.0119
Episode:1350 meanR:48.9500 R:59.0000 rate:0.1180 aloss:0.5712 eloss:4.2274 aloss2:2.1946 exploreP:0.0118
Episode:1351 meanR:48.7400 R:32.0000 rate:0.0640 aloss:0.5522 eloss:4.2202 aloss2:2.1924 exploreP:0.0118
Episode:1352 meanR:48.6400 R:55.0000 rate:0.1100 aloss:0.5217 eloss:4.2293 aloss2:2.2021 exploreP:0.0118
Episode:1353 meanR:48.5800 R:34.0000 rate:0.0680 aloss:

Episode:1423 meanR:47.5300 R:78.0000 rate:0.1560 aloss:0.5687 eloss:4.2238 aloss2:2.2002 exploreP:0.0113
Episode:1424 meanR:47.7300 R:64.0000 rate:0.1280 aloss:0.5051 eloss:4.2416 aloss2:2.1925 exploreP:0.0113
Episode:1425 meanR:47.5700 R:40.0000 rate:0.0800 aloss:0.5235 eloss:4.2293 aloss2:2.1992 exploreP:0.0113
Episode:1426 meanR:47.8700 R:66.0000 rate:0.1320 aloss:0.5029 eloss:4.2429 aloss2:2.2011 exploreP:0.0113
Episode:1427 meanR:47.7700 R:38.0000 rate:0.0760 aloss:0.5874 eloss:4.2429 aloss2:2.1842 exploreP:0.0113
Episode:1428 meanR:47.7100 R:69.0000 rate:0.1380 aloss:0.5290 eloss:4.2376 aloss2:2.1935 exploreP:0.0113
Episode:1429 meanR:47.9800 R:61.0000 rate:0.1220 aloss:0.5000 eloss:4.2367 aloss2:2.1837 exploreP:0.0113
Episode:1430 meanR:48.3600 R:74.0000 rate:0.1480 aloss:0.5035 eloss:4.2495 aloss2:2.1692 exploreP:0.0112
Episode:1431 meanR:48.6100 R:61.0000 rate:0.1220 aloss:0.5284 eloss:4.2362 aloss2:2.1948 exploreP:0.0112
Episode:1432 meanR:48.8200 R:71.0000 rate:0.1420 aloss:

Episode:1502 meanR:47.8300 R:44.0000 rate:0.0880 aloss:0.4979 eloss:4.2351 aloss2:2.1890 exploreP:0.0109
Episode:1503 meanR:48.0200 R:87.0000 rate:0.1740 aloss:0.5040 eloss:4.2364 aloss2:2.1978 exploreP:0.0109
Episode:1504 meanR:47.7800 R:36.0000 rate:0.0720 aloss:0.5562 eloss:4.2281 aloss2:2.1875 exploreP:0.0109
Episode:1505 meanR:47.7900 R:36.0000 rate:0.0720 aloss:0.5103 eloss:4.2308 aloss2:2.1997 exploreP:0.0109
Episode:1506 meanR:47.9500 R:50.0000 rate:0.1000 aloss:0.4989 eloss:4.2432 aloss2:2.1980 exploreP:0.0109
Episode:1507 meanR:47.7700 R:34.0000 rate:0.0680 aloss:0.5267 eloss:4.2333 aloss2:2.1868 exploreP:0.0109
Episode:1508 meanR:47.3900 R:38.0000 rate:0.0760 aloss:0.5579 eloss:4.2326 aloss2:2.1883 exploreP:0.0109
Episode:1509 meanR:47.5200 R:52.0000 rate:0.1040 aloss:0.5107 eloss:4.2339 aloss2:2.1889 exploreP:0.0109
Episode:1510 meanR:47.4100 R:36.0000 rate:0.0720 aloss:0.4875 eloss:4.2335 aloss2:2.1935 exploreP:0.0109
Episode:1511 meanR:47.5700 R:51.0000 rate:0.1020 aloss:

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
